In [1]:
import matplotlib.pyplot as plt
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import nibabel as nib
from scipy import ndimage
import glob
import sys,os
from natsort import natsorted
import tensorflow_probability as tfp
import random
from aifnet_utils.preprocess import read_nifti_file, normalize, normalize_aif, process_scan
from aifnet_utils.losses import MaxCorrelation
from aifnet_utils.data_loaders import read_isles_annotations, read_isles_volumes

%matplotlib inline

In [19]:
#Dataset generator
class ISLES18DataGen_aifvof(tf.keras.utils.Sequence):
  
    def __init__(self, 
                 ctp_volumes,
                 annotations_aif,
                 annotations_vof,
                 minimum_number_volumes_ctp,
                 batch_size=1,
                 input_size=(256, 256, None,43),
                 time_arrival_augmentation = True,
                 delay_t = None,
                 shuffle=True):
        self.ctp_volumes = ctp_volumes, 
        self.labels_aif = annotations_aif,
        self.labels_vof = annotations_vof,
        self.minimum_number_volumes_ctp = minimum_number_volumes_ctp,
        self.batch_size = batch_size,
        self.delay_t = delay_t,
        self.input_size = input_size,
        self.shuffle = shuffle,
        self.augment = time_arrival_augmentation,
        self.n = len(self.ctp_volumes),
        self.indices = np.arange(len(self.ctp_volumes))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __get_input(self, img_idx):
        #Get the volume
        ctp_vals = self.ctp_volumes[img_idx]['ctpvals']
        volume_sequence = normalize(ctp_vals)
        #Get the labels
        case_id = ctp_volumes[img_idx]['image'].split('.')[-2]

        label_aif = normalize_aif(self.labels_aif[0][case_id])
        label_vof = normalize_aif(self.labels_vof[0][case_id])
        labels = [label_aif,label_vof]
        #labels = np.array([label_aif,label_vof])
        if self.augment:
            augment_functions = [early_bolus,late_bolus]
            random_augmentation = random.choice(augment_functions)
            if delay_t == None:
                delay_t = np.random.randint(0,10)
            volume, labels = random_augmentation(volume_sequence,[label_aif,label_vof], delay_t)
        return volume,labels

    
    def __getitem__(self, idx): #This function returns the batch 
        print(self.indices)
        print(len(self.ctp_volumes))
        print(self.ctp_volumes)
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        #print(inds)
        #batch_x = [self.ctp_volumes[index] for index in inds]
        #batch_y = self.annotations[inds]
        batch_x, batch_y = [], []
        for index in inds:            
            x, y = self.__get_input(index)
            batch_x.append(x)
            batch_y.append(y)
        return np.array(batch_x), np.array(batch_y).squeeze()

    def __len__(self):
        return self.n // self.batch_size

    ##CPT Augmentation techniques for including in the data loaders
    def delay_sequence_padding(sequence,delay_t):
        nb_timepoints = np.array(sequence).shape[0] #This assumes a SINGLE vector with all the timepoints
        delayed_sequence = np.zeros(np.array(sequence).shape)
        delayed_sequence[0:delay_t] = sequence[0] #Repeating the first time point
        delayed_sequence[delay_t:]  = sequence[1:nb_timepoints-delay_t+1]
        return delayed_sequence

    def anticipate_sequence_padding(sequence,delay_t):
        nb_timepoints = np.array(sequence).shape[0] #This assumes a SINGLE vector with all the timepoints
        early_intensity = np.zeros(sequence.shape)
        early_intensity[0:nb_timepoints-delay_t] = sequence[delay_t:nb_timepoints] #Shifting the first time points
        early_intensity[nb_timepoints-delay_t:] = sequence[-1]
        return early_intensity


    def late_bolus(volume_sequence, labels, delay_t=None):
        labels_shape = np.array(labels).shape
            
        delayed_volume, delayed_intensity = np.zeros(volume_sequence.shape), np.zeros(np.array(labels).shape)
        nb_timepoints = volume_sequence.shape[-1]

        first_volume = volume_sequence[:,:,:,0]    
        if delay_t == None:
            delay_t = random.randint(1,int(nb_timepoints/3))
        if delay_t == 0:
            return volume_sequence, labels
        
        for i in range(0,delay_t+1):
            delayed_volume[:,:,:,i] = first_volume
        #print(i)
        delayed_volume[:,:,:,i:] = volume_sequence[:,:,:,1:nb_timepoints-i+1]
        
        #Delaying each of the labels
        if len(labels_shape)==2:#We are processing the AIF and the VOF
            delayed_aif = delay_sequence_padding(labels[0],delay_t)
            delayed_vof = delay_sequence_padding(labels[1],delay_t)
            delayed_intensity = np.array([delayed_aif,delayed_vof])
        else: #We are only processing the AIF
            delayed_intensity = delay_sequence_padding(labels,delay_t)

        return delayed_volume, delayed_intensity

    def early_bolus(volume_sequence, labels, delay_t=None):
        early_volume, early_intensity = np.zeros(volume_sequence.shape), np.zeros(labels.shape)
        nb_timepoints = volume_sequence.shape[-1]
        labels_shape = np.array(labels).shape

        last_volume = volume_sequence[:,:,:,-1]
        if delay_t == None:
            delay_t = random.randint(1,int(nb_timepoints/3))
        if delay_t == 0:
            return volume_sequence, labels
        
        early_volume[:,:,:,0:nb_timepoints-delay_t] = volume_sequence[:,:,:,delay_t:nb_timepoints]
        for i in range(nb_timepoints-delay_t,nb_timepoints):   
            early_volume[:,:,:,i] = volume_sequence[:,:,:,-1]
        if len(labels_shape)==2:#We are processing the AIF and the VOF
            early_aif = anticipate_sequence_padding(labels[0],delay_t)
            early_vof = anticipate_sequence_padding(labels[1],delay_t)
            early_intensity = np.array([early_aif,early_vof])
        else: #We are only processing the AIF
            early_intensity = anticipate_sequence_padding(labels,delay_t)

        return early_volume, early_intensity



In [20]:
keras.backend.set_image_data_format('channels_last')

In [4]:
#Dataset generator
class ISLES18DataGen_aifvof_old(tf.keras.utils.Sequence):
  
    def __init__(self, 
                 ctp_volumes,
                 annotations_aif,
                 annotations_vof,
                 minimum_number_volumes_ctp,
                 batch_size=1,
                 input_size=(256, 256, None,43),
                 shuffle=True):
        self.ctp_volumes = ctp_volumes 
        self.labels_aif = annotations_aif,
        self.labels_vof = annotations_vof,
        self.minimum_number_volumes_ctp = minimum_number_volumes_ctp
        self.annotations = [annotations_aif],[annotations_vof]
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle        
        self.n = len(self.ctp_volumes)
        self.indices = np.arange(len(self.ctp_volumes))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __get_input(self, img_idx):
        #Get the volume
        ctp_vals = self.ctp_volumes[img_idx]['ctpvals']
        volume = normalize(ctp_vals)
        #Get the labels
        case_id = ctp_volumes[img_idx]['image'].split('.')[-2]

        label_aif = normalize_aif(self.labels_aif[0][case_id])
        label_vof = self.labels_vof[0][case_id]
        #labels = np.array([label_aif,label_vof])
        return volume,[label_aif,label_vof]
        #return volume,label_aif
    
    def pct_augment():
        pass
    
    def __getitem__(self, idx): #This function returns the batch 
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        #print(inds)
        #batch_x = [self.ctp_volumes[index] for index in inds]
        #batch_y = self.annotations[inds]
        batch_x, batch_y = [], []
        for index in inds:            
            x, y = self.__get_input(index)
            batch_x.append(x)
            batch_y.append(y)
        return np.array(batch_x), np.array(batch_y).squeeze()

    def __len__(self):
        return self.n // self.batch_size

In [6]:
pp_late, pl_late   = late_bolus(pp,[aif_annotations['345582'],aif_annotations['345582']],20)
pp_early, pl_early = early_bolus(pp,aif_annotations['345582'],20)

NameError: name 'late_bolus' is not defined

In [20]:
for i in range(43):
    print(str(i) + " <=> " + str(int(np.sum(pp[:,:,:,i].flatten())))  + ", " + str(int(np.sum(pp_early[:,:,:,i].flatten())))
         + "   ==  " +str((aif_annotations['345582'][i])) +", " + str((pl_early[i])))

0 <=> 17446955, 18274742   ==  46.00687026977539, 64.83368682861328
1 <=> 17491673, 18187071   ==  45.46455383300781, 68.8190689086914
2 <=> 17497241, 18154510   ==  44.42026138305664, 63.13859558105469
3 <=> 17516392, 18138570   ==  51.08286666870117, 68.21202850341797
4 <=> 17535508, 18099539   ==  72.60696411132812, 59.45867156982422
5 <=> 17697656, 18098316   ==  100.85197448730469, 92.30133819580078
6 <=> 17921554, 18347820   ==  122.21617889404297, 79.43942260742188
7 <=> 18159512, 18081879   ==  137.44642639160156, 79.08687591552734
8 <=> 18370063, 18094479   ==  119.1580810546875, 53.11215591430664
9 <=> 18637204, 18108415   ==  172.7520751953125, 69.41795349121094
10 <=> 18880359, 18087250   ==  163.13604736328125, 65.49166107177734
11 <=> 19055675, 18068658   ==  147.68309020996094, 73.1170883178711
12 <=> 19117529, 18054814   ==  137.92901611328125, 77.52625274658203
13 <=> 19151811, 18051509   ==  117.84191131591797, 71.06597137451172
14 <=> 19108125, 18077758   ==  103.170

In [21]:
for i in range(43):
    print(str(i) + " <=> " + str(int(np.sum(pp[:,:,:,i].flatten())))  + ", " + str(int(np.sum(pp_late[:,:,:,i].flatten())))
         + "   ==  " +str((aif_annotations['345582'][i])) +", " + str((pl_late[i])))

0 <=> 17446955, 17446955   ==  46.00687026977539, [ 46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  45.46455383  44.42026138  51.08286667  72.60696411 100.85197449
 122.21617889 137.44642639 119.15808105 172.7520752  163.13604736
 147.68309021 137.92901611 117.84191132 103.17086792  94.40621948
  87.79454041  64.13690948  65.01002502  64.1909256   64.83368683
  68.81906891  63.13859558  68.2120285 ]
1 <=> 17491673, 17446955   ==  45.46455383300781, [ 46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  46.00687027  46.00687027  46.00687027  46.00687027  46.00687027
  45.46455383  44.42026138  51.08286667  72.60696411 100.85197449
 

IndexError: index 2 is out of bounds for axis 0 with size 2

In [22]:
43-10

33

In [23]:
pp_late
np.sum(np.sqrt(normalize(pp_late[0]).flatten()))

0.0

In [7]:
def scale_pct_values(pre_contrast,pct_sequence):
    scaled_sequence = np.zeros(pct_sequence.shape)
    #Substracting the pre-contrast to the sequence
    for i in range(nb)
    
    return scaled_sequence

SyntaxError: invalid syntax (<ipython-input-7-29d5b120a46a>, line 4)

In [49]:
scale_pct_values(pre_contrast=None,pct_sequence=pp)

NameError: name 'scale_pct_values' is not defined

In [9]:
root_dir     = '/media/sebastian/data/ASAP/ISLES2018_Training'
#At insel: /media/sebastian/data/ASAP/ISLES2018_Training
#Local: '/Users/sebastianotalora/work/postdoc/data/ISLES/'
aif_annotations_path = '/home/sebastian/experiments/aifnet_replication/annotated_aif_vof_complete.csv'
min_num_volumes_ctp = 43
ROOT_EXP = '/home/sebastian/experiments/aifnet_replication'#'/Users/sebastianotalora/work/postdoc/ctp/aifnet_replication'


In [10]:
aif_annotations, vof_annotations = read_isles_annotations(aif_annotations_path, root_dir, 
                                         min_num_volumes_ctp, return_aif_only = False)

In [11]:
ctp_volumes = read_isles_volumes(root_dir, aif_annotations_path, min_num_volumes_ctp, take_two_slices_only=False)

In [32]:
ctp_volumes[6]

{'image': '/media/sebastian/data/ASAP/ISLES2018_Training/TRAINING/case_8/SMIR.Brain.XX.O.CT_4DPWI.345610/SMIR.Brain.XX.O.CT_4DPWI.345610.nii',
 'ctpvals': array([[[[-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          [-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          [-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          ...,
          [-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          [-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          [-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ]],
 
         [[-23.        , -23.        , -23.        , ..., -23.        ,
           -23.        , -23.        ],
          [-23.        , -23.        , -23. 

In [17]:
len(vof_annotations)

93

In [21]:
train_datagen = ISLES18DataGen_aifvof(ctp_volumes=ctp_volumes, annotations_aif=aif_annotations,
                               annotations_vof=vof_annotations,minimum_number_volumes_ctp = 43, batch_size=1)

In [22]:
train_datagen.__getitem__(1)

[0]
1
([{'image': '/media/sebastian/data/ASAP/ISLES2018_Training/TRAINING/case_1/SMIR.Brain.XX.O.CT_4DPWI.345561/SMIR.Brain.XX.O.CT_4DPWI.345561.nii', 'ctpvals': array([[[[-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         ...,
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.]],

        [[-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         ...,
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.]],

        [[-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         [-23., -23., -23., ..., -23., -23., -23.],
         ...,
         [-2

TypeError: slice indices must be integers or None or have an __index__ method

In [87]:
delay_sequence_padding

<function __main__.delay_sequence_padding(sequence, delay_t)>

In [50]:
early_bolus

<function __main__.early_bolus(volume_sequence, labels, delay_t=None)>